<h2><center>IIC2440 – Procesamiento de Datos Masivos</center></h2>
<h3><center>Tarea 1: Análisis sesiones parlamentarias</center></h3>


**Integrantes:**
- Franco Chiappe
- Tomás Pérez

### Librerias

In [21]:
# Importar librerias 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import unicodedata

## Parte 1 [1 pto] - Procesando los datos

In [22]:
# Cargar los datos
parlamentarios_info = pd.read_csv('participacion/parlamentarios_info_general.csv')
descripcion_2023_1 = pd.read_csv('participacion/participacion_descripcion_2023_1.csv')
descripcion_2023_2 = pd.read_csv('participacion/participacion_descripcion_2023_2.csv')
descripcion_2024 = pd.read_csv('participacion/participacion_descripcion_2024.csv')
df = pd.concat([descripcion_2023_1, descripcion_2023_2, descripcion_2024], ignore_index=True)

### Pre-Procesamiento de Datos

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68161 entries, 0 to 68160
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_PARTICIPACION    68161 non-null  int64  
 1   PARLAMENTARIO_ID    68161 non-null  int64  
 2   FECHA               68161 non-null  object 
 3   LEGISLATURA         68161 non-null  int64  
 4   SESION              68161 non-null  int64  
 5   TIPO_SESION         68161 non-null  object 
 6   TIPO_PARTICIPACION  68161 non-null  object 
 7   CAMARA              68161 non-null  object 
 8   PARLAMENTARIOS      68161 non-null  object 
 9   TEXTO_PRINCIPAL     68161 non-null  object 
 10  TEXTO_ANTECEDENTE   32393 non-null  object 
 11  DESCRIPCION_DEBATE  68161 non-null  object 
 12  BOLETIN_ID          20593 non-null  float64
dtypes: float64(1), int64(4), object(8)
memory usage: 6.8+ MB


Nos damos cuenta que en las columnas:

- `TEXTO_ANTECEDENTE` tiene 35768  datos nulos
- `BOLETIN_ID` tiene 47568 datos nulos

Decidimos eliminar la columna `TEXTO_ANTECEDENTE`, porque presentaba un alto porcentaje de datos nulos y no aportaba informacion relevante. Por otro parte, no eliminamos la columna `BOLETIN_ID`, esta columna podria resultar util para analisis futuros.

In [24]:
df = df.drop(columns=["TEXTO_ANTECEDENTE"])

### Limpieza de texto

In [25]:
def limpiar_texto(texto):
        
    texto = texto.lower()
    texto = texto.strip()
    
    texto = re.sub(r'[^\w\s]', '', texto)
    texto = re.sub(r'\s+', ' ', texto)

    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')

    return texto

Se usa la funcion ***limpiar_texto*** el objetivo es normalizar los textos para facilitar la extraccion de keywords. Esta funcion convierte todo el contenido a minuscula, elimina tildes, signos de puntuación y espacios innecesarios, dejando un text limpio. Esto permite detectar con mayor precision los temas.

In [26]:
# Aplicar la funcion de limpieza a las columnas
df['DESCRIPCION_DEBATE'] = df['DESCRIPCION_DEBATE'].apply(limpiar_texto)
df['TEXTO_PRINCIPAL'] = df['TEXTO_PRINCIPAL'].apply(limpiar_texto)

# Juntamos las columnas
df["TEXTO_COMBINADO"] = df["TEXTO_PRINCIPAL"] + " " + df["DESCRIPCION_DEBATE"]
df["TEXTO_COMBINADO"].head()

0    la senora allende ngracias presidente npor su ...
1    la senora allende ngracias presidente nen prim...
2    proyecto de acuerdo de los honorables senadore...
3    de la senora allende n a la municipalidad de v...
4    proyecto de acuerdo de los honorables senadore...
Name: TEXTO_COMBINADO, dtype: object

### Temas y palabras claves

In [27]:
temas = {
    "salud": ["salud", "hospital", "hospitales" "medico", "vacuna", "enfermedad", "medicina", "medicamento"],
    "educacion": ["educacion", "colegio", "escuela", "universidad", "profesor", "alumno", "estudiante", "educador", "colegios"],
    "delincuencia": ["delincuencia", "carabinero", "robo", "narcotrafico", "seguridad", "delito", "delincuente", "delincuentes"],
    "medio ambiente": ["medioambiente", "naturaleza", "contaminacion", "reciclaje", "agua", "bosque", "bosques", "flora", "fauna", "ecosistema", "ecosistemas"],
    "economia": ["economia", "dinero", "impuesto", "trabajo", "impuestos", "empresa", "empresas", "negocio", "negocios", "inversion", "inversiones", "financiero", "financieros"],
    "constitucion": ["constitucion", "constitucional"],
    "vivienda": ["vivienda", "casa", "alquiler", "arrendar"],
    "transporte": ["transporte", "metro", "camino", "micro"],
    "derechos humanos": ["derechos humanos", "derechos", "humanos", "discriminacion", "discriminacion", "discriminados", "discriminados", "discriminacion", "discriminaciones", "humanitaria"],
    "legislacion": ["legislacion", "ley", "leyes", "legislativo", "legislativa"],
    "justicia": ["justicia", "justicia", "jueces", "juez", "tribunal", "tribunales"],
}

In [28]:
def asignar_keywords(texto):
    palabras = texto.split()
    etiquetas = set()
    for tema, claves in temas.items():
        for palabra in claves:
            if (palabra in palabras):
                etiquetas.add(tema)
       
    return list(etiquetas)

In [29]:
# Se agrupan por descripción debate, para poder aplicar la función de asignar_keywords a cada grupo de manera más eficiente
grouped_df = df.groupby("DESCRIPCION_DEBATE").first().reset_index()
grouped_df["KEYWORDS"] = grouped_df["TEXTO_COMBINADO"].apply(asignar_keywords)

df = df.merge(grouped_df[["TEXTO_COMBINADO", "KEYWORDS"]], on="TEXTO_COMBINADO", how="left")

### Conclusion:



Se realizo el preprocesamiento de las intervenciones parlamentarias con el objetivo de identificar las tematicas. Para esto, se aplico una funcion de limpieza que normalizo el texto (conversion a minusculas, eliminacion de tildes, puntuacion y espacios innecesarios). Luego, se unieron las columnas `TEXTO_ANTECEDENTE` y `DESCRIPCION_DEBATE` para generar un texto combinado. Finalmente, se aplico una funcion que detecta las palabras clave y asigna a cada intervencion una o mas temas, permitiendo categorizar los discursos parlamentarios segun temas como salud, educación, delincuencia, entre otros. Para optimizar la asignación, se agruparon las filas por la columna `DESCRIPCION_DEBATE`, ya que muchas compartian o eran de un mismo debate, para así acortar el tiempo de ejecución de asignación

## Parte 2 - [1 pts] - Modelamiento

### Desarrollo en **P2_T1_Datos_Masivos.pdf**

A Continuación se encuentra el codigo con el que se trabajo para generar las tablas

## Crear DataFrames, para trabajar estos posteriormente como tablas

<h3> Creamos el DataFrame Parlamentarios </h3>

In [30]:
parlamentarios_info["PARTIDO_POLITICO"] = parlamentarios_info["PARTIDO_POLITICO"].fillna("Independiente")
parlamentarios = parlamentarios_info.copy()
parlamentarios = parlamentarios.drop(columns=["REDES_SOCIALES", "SITIOS_WEB", "PARTIDO_POLITICO"])

<h3> Creamos el DataFrame Partidos_Politico  </h3>

In [31]:
partidos = parlamentarios_info[["PARTIDO_POLITICO"]].drop_duplicates().reset_index(drop=True)
partidos["PARTIDO_ID"] = range(1, len(partidos) + 1)
partidos.columns = ["PARTIDO_POLITICO", "PARTIDO_ID"]
partido_to_id = partidos.set_index("PARTIDO_POLITICO")["PARTIDO_ID"].to_dict()

<h3> Creamos el DataFrame  Debates </h3>


In [32]:
debates = df.copy()
debates["DEBATE_ID"] = debates.groupby(["DESCRIPCION_DEBATE"]).ngroup() + 1
debates["BOLETIN_ID"] = debates["BOLETIN_ID"].fillna(0).astype(int)

debates = debates[["DEBATE_ID", "DESCRIPCION_DEBATE", "FECHA", "BOLETIN_ID"]]
debates = debates.drop_duplicates(subset=["DESCRIPCION_DEBATE"])

debates = debates.sort_values(by="DEBATE_ID").reset_index(drop=True)

<h3> Creamos el DataFrame Participaciones </h3>


In [33]:
participaciones = df.copy()

debate_to_id = debates.set_index("DESCRIPCION_DEBATE")["DEBATE_ID"].to_dict()
participaciones["DEBATE_ID"] = participaciones["DESCRIPCION_DEBATE"].map(debate_to_id)

participaciones = participaciones[[ "ID_PARTICIPACION", "PARLAMENTARIO_ID", "DEBATE_ID", "FECHA", "LEGISLATURA", "SESION", "TIPO_SESION", "TIPO_PARTICIPACION", "CAMARA"]]

participaciones = participaciones.sort_values(by="ID_PARTICIPACION").reset_index(drop=True)

<h3> Creamos el DataFrame Keywords </h3>


In [34]:
keywords = pd.DataFrame(list(temas.items()), columns=["Tema", "Palabras_Clave"])
keywords.rename(columns={"Tema": "KEYWORD"}, inplace=True)
keywords.drop(columns=["Palabras_Clave"], inplace=True)
keywords["KEYWORD_ID"] = range(1, len(keywords) + 1)

<h3> Creamos el DataFrame Intervenciones </h3>


In [35]:
intervenciones = df[["ID_PARTICIPACION", "TEXTO_PRINCIPAL"]].copy()
intervenciones["INTERVENCION_ID"] = range(1, len(intervenciones) + 1)
intervenciones = intervenciones[["INTERVENCION_ID", "ID_PARTICIPACION", "TEXTO_PRINCIPAL"]]
intervenciones = intervenciones.sort_values(by="INTERVENCION_ID").reset_index(drop=True)

<h3> Creamos el DataFrame Intervenciones_Keywords </h3>


In [36]:
df_intervenciones_keywords = df[["ID_PARTICIPACION", "KEYWORDS"]].explode("KEYWORDS").dropna(subset=["KEYWORDS"])

id_participacion_to_intervencion_id = intervenciones.set_index("ID_PARTICIPACION")["INTERVENCION_ID"].to_dict()
df_intervenciones_keywords["INTERVENCION_ID"] = df_intervenciones_keywords["ID_PARTICIPACION"].map(id_participacion_to_intervencion_id)

keyword_to_id = keywords.set_index("KEYWORD")["KEYWORD_ID"].to_dict()
df_intervenciones_keywords["KEYWORD_ID"] = df_intervenciones_keywords["KEYWORDS"].map(keyword_to_id)

df_intervenciones_keywords = df_intervenciones_keywords[["INTERVENCION_ID", "KEYWORD_ID"]].dropna().drop_duplicates()

<h3> Guardamos los DataFrames </h3>

In [37]:
import zipfile

with zipfile.ZipFile("DataFrames.zip", "w") as zipf:

    parlamentarios.to_csv("Parlamentarios.csv", index=False)
    zipf.write("Parlamentarios.csv")
    
    partidos.to_csv("Partidos_Politico.csv", index=False)
    zipf.write("Partidos_Politico.csv")
    
    debates.to_csv("Debates.csv", index=False)
    zipf.write("Debates.csv")
    
    participaciones.to_csv("Participaciones.csv", index=False)
    zipf.write("Participaciones.csv")
    
    keywords.to_csv("Keywords.csv", index=False)
    zipf.write("Keywords.csv")
    
    intervenciones.to_csv("Intervenciones.csv", index=False)
    zipf.write("Intervenciones.csv")
    
    df_intervenciones_keywords.to_csv("Intervenciones_Keywords.csv", index=False)
    zipf.write("Intervenciones_Keywords.csv")


In [38]:
import os

archivos = [
    "Parlamentarios.csv",
    "Partidos_Politico.csv",
    "Debates.csv",
    "Participaciones.csv",
    "Keywords.csv",
    "Intervenciones.csv",
    "Intervenciones_Keywords.csv"
]

for archivo in archivos:
    if os.path.exists(archivo):
        os.remove(archivo)


## Parte 4 - [2 pts] - Informe

### Desarrollo en  **P4_T2_Datos_Masivos.pdf**

## Parte 5 - [0.5 pts] - Resumen ejecutivo